# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.  
Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Нужно построить модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
import numpy as np

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df['calls'].unique()

array([ 40.,  85.,  77., 106.,  66.,  58.,  57.,  15.,   7.,  90.,  82.,
        45.,  51.,  56., 108.,   6.,   2.,  26.,  79.,  49.,  93.,  48.,
        11.,  53.,  81., 154.,  37.,  50.,  41.,  10.,  71.,  65., 110.,
       120.,  76.,  64.,  23.,  34.,  98.,  35.,   5.,  70., 124., 129.,
        67.,   0.,  13.,  68.,  91., 121., 114., 125.,  80.,  33., 138.,
        84.,  78.,  69.,  63.,  72.,  73.,   1.,  43., 118.,  74.,  83.,
       141., 117.,  54., 101.,  29.,   3., 107.,  55.,  47., 158.,  87.,
        28.,  59.,  52.,  44.,  17., 111., 109.,  14.,  92.,  94.,  46.,
       133.,  75.,  38.,  60., 100.,  31.,  61.,  89.,  27., 196.,  24.,
        99.,  62., 162., 116., 123.,  18.,  21.,  12.,  86.,  32.,  95.,
        39.,  30.,  25.,  36.,  42., 113.,   9., 183., 156., 127.,  96.,
        16.,   4., 102.,  97.,  20., 104., 144.,  19., 132., 131., 136.,
        88., 115., 176., 160., 164., 169.,  22., 105., 152., 177., 161.,
       112.,   8., 126., 178., 103., 130., 198., 11

In [7]:
df['is_ultra'].unique()

array([0, 1])

In [8]:
df['calls'] = df['calls'].astype(int)

In [9]:
df['is_ultra'] = df['is_ultra'].astype(bool)

In [13]:
df.sample(3)

,calls,minutes,messages,mb_used,is_ultra
1007,73,550.58,25.0,18322.63,False
1774,35,232.86,80.0,19193.08,False
1645,97,664.39,36.0,18982.03,False


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73


Датасет содержит информацию о 3213 абонентах сети Мегалайн. Каждый абонент характеризуется набором признаков:  
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

Пропусков в данных нет. Провели легкую предобработку - для признака "сalls" заменили тип данных float на int, для "is_ultra" - int на bool.

## Разбейте данные на выборки

In [15]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [23]:
features_train, features_valid, target_train, target_valid = \
train_test_split(features, target, test_size=.4, random_state=12, stratify = target)

In [25]:
features_valid, features_test,  target_valid, target_test = \
train_test_split(features_valid, target_valid, test_size=.5, random_state=12, stratify = target_valid)

Для дальнейшей разработки моделей данные были разбиты на 3 группы - обучающую (df_train), тестовую (df_test) и валидационную (df_valid) выборки. Соотношение обучающей, тестовой и валидационной выборки - 3:1:1.

In [31]:
features_valid.shape[0] / features_test.shape[0]

1.0

In [32]:
features_train.shape[0] / features_test.shape[0]

2.998444790046656

## Исследуйте модели

На данном этапе попытаемся спрогнозировать значение катериального параметра is_ultra с помощью трех моделей: решающих деревьев, случайного леса и логистической регрессии. Начнем с деревьев.

In [37]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12, max_depth=depth) # обучите модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучите модель
    predictions = model.predict(features_valid) # получите предсказания модели
    result = accuracy_score(target_valid, predictions) # посчитайте качество модели
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
best_tree_model = best_model
print("Accuracy лучшей модели:", best_result)        
print("depth лучшей модели:", best_depth)        

Accuracy лучшей модели: 0.7667185069984448
depth лучшей модели: 3


Модель дерева решений с гиперпараметром depth = 3 выдает точность accuracy около 77% на валидационной выборке. Неплохо. Проверим случайный лес.

In [41]:
best_model = None
best_result = 0
for est in range(1, 92, 5):
    for depth in range(1, 30):
        model = RandomForestClassifier(random_state=12, n_estimators=est, max_depth=depth) # обучите модель с заданным количеством деревьев
        model.fit(features_train, target_train) # обучите модель на тренировочной выборке
        result = model.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
        if result > best_result:
            best_model = model# сохраните наилучшую модель
            best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных
            best_est = est
            best_depth = depth
best_forest_model = best_model
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Количество деревьев в наилучшей модели:", best_est)
print("Глубина деревьев в наилучшей модели:", best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8055987558320373
Количество деревьев в наилучшей модели: 21
Глубина деревьев в наилучшей модели: 11


Качество классификации с омощью случайного леса немного выше - около 80%. При этом в лучшем лесу 21 дерево с глубиной 11.    
Перейдем к логистической регрессии.

In [43]:
model = LogisticRegression(random_state=12, solver = 'liblinear') 
model_log_regressor = model.fit(features_train, target_train) 
result = model_log_regressor.score(features_valid, target_valid) 
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.6998444790046656


Неожиданно низкая точность у логистической регрессии - всего 70% на валидационной выборке. Возможно, линейный классификатор попросту не подходит для разбиения наших данных на два класса.  

Итак - в лидерах случайный лес (точность 80%), немного хуже справляется модель дерева (77%) и совсем плохо - лог. регрессия (всего 70%). 

## Проверьте модель на тестовой выборке

In [44]:
best_tree_model.fit(features_train.append(features_valid), target_train.append(target_valid))
predictions = best_tree_model.predict(features_test) 
result = accuracy_score(target_test, predictions) 
result

0.7807153965785381

In [45]:
best_forest_model.fit(features_train.append(features_valid), target_train.append(target_valid))
predictions = best_forest_model.predict(features_test) 
result = accuracy_score(target_test, predictions) 
result

0.7993779160186625

Точность модели случайного леса на тестовой выборке оказалась на примерно как на валидационной и осталась чуть выше, чем для дерева.

In [46]:
model_log_regressor.fit(features_train.append(features_valid), target_train.append(target_valid))
predictions = model_log_regressor.predict(features_test) 
result = accuracy_score(target_test, predictions) 
result

0.7402799377916018

Точность логистической регрессии на тестовой выборке оказалась чуть ниже, чем на валидационной. Тем не менее, данная модель по-прежнему самая грубая для представленных данных.  

Общий вывод после проверки на тестовой выборке немного скорректировался - теперь модель дерева решений сравнялась со случайным лесом.

 А теперь с учетом дообучения лог. регрессия стала точнее, но все равно уступает остальным моделям. 

## (бонус) Проверьте модели на адекватность

Попробуем построить модель, которая случайно угадывает тариф.  
Для начала найдем долю пользователей Ультра.

In [47]:
df['is_ultra'].mean()

0.30647168637212197

Доля Ультра - около 30%.

In [49]:
predictions = np.random.rand(len(features_test)) <= .3

In [50]:
result = accuracy_score(target_test, predictions) 
result

0.5925349922239502

In [55]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train.append(features_valid), target_train.append(target_valid))
predictions = dummy_clf.predict(features_test)
result = accuracy_score(target_test, predictions) 
result

0.6936236391912908

Получаем, если создать совершенно бестолковую модель, которая вообще не анализирует никакие параметры абонентов, а лишь знает, что 30% из них в итоге выбрали Ультра и  на этом основании случайно случайно их классифицирует, точность такой классификации будет 56%.  
То есть 56% в данном случае можно рассматривать как некую "красную черту", которая задает самый нижний уровень точности. Все три модели показали более высокую точность, значит они все обладают той или иной практической ценностью в данной задаче. 
Простое присвоение наиболее вероятного класса целевому признаку дает точность около 70%, и расчеты через DummyClassifier это подтвердили. Даже логистичнская регрессия на тестовой выборке дает более точный результат.
